### GENERATING FEATURES

In [ ]:
!pip3 install scikit-image

In [ ]:
import library
import color_features
import texture_features
import glcm_features
import hair_removal
import pandas as pd
import numpy as np
import cv2
import os

samples = library.get_sample(path = "/home/name/Desktop/CAD/challenge1/train", amount=3000)

dictF = {}
features = pd.DataFrame()
count = 0
flag = True
for sample in samples:
    print('count ', count)
    count += 1
    img = cv2.imread(sample)
    output_bh = library.hair_removal_BH(img)
    
    dictF['name'] = sample
    dictF['label'] = (0 if 'nevus' in sample else 1 )
    
    # color features
    colors = color_features.extract_color_features(output_bh)
    
    dictF.update(colors)
    
    #glcm features
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    distances = [1]
    colorspaces = ['rgb', 'hsv', 'lab', 'ycc', 'gray']

    for cs in colorspaces:
        glcm = glcm_features.get_glcm(output_bh, angles, distances, cs)
        dictF.update(glcm)
    
    # lbp features
    lbp = texture_features.extract_lbp(output_bh, 1, 8)
    dictF.update(lbp)
    
    # orb features
    # orb = texture_features.extract_orb(output_bh, 64)
    # dictF.update(orb)
    
    features = features.append(dictF, ignore_index=True)
    
    library.writeFeatures(features,
                  flag,
                  os.path.join('/home',
                             'name',
                             'Desktop',
                             'CAD'),
                  'features_train_bh_3000.csv')
    
    flag = False
    features = pd.DataFrame()
    dictF.clear()

In [ ]:
from sklearn.model_selection import train_test_split
import library
from sklearn import tree
import pandas as pd
import os
    

classifiers = ["rf", "tree", "svm", "adaboost", "gradboost", "knn"]

train = pd.read_csv(os.path.join('/home','emily','Desktop','CAD','features_briashaver_colors_glcm_854.csv'))

y = train['label']
X = train.drop(['label'], axis=1)
X = X.drop(['name'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print("x_test ", len(X_test))
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

for classifier in classifiers:
    
    if classifier == "svm":
        clf, best_params = library.SVC_linear(X_val, y_val, cv=2)
        clf.set_params(**best_params) 
        print("### SVM ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "rf":
        clf, best_params = library.RandomForest(X_val, y_val, cv=2)
        clf.set_params(**best_params)
        print("### RF ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "tree":
        clf = tree.DecisionTreeClassifier()
        print("### TREE ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "adaboost":
        clf, best_params = library.AdaBoost(X_val, y_val)
        clf.set_params(**best_params)
        print("### ADABOOST ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "gradboost":
        clf, best_params = library.GradientBoosting(X_val, y_val)
        clf.set_params(**best_params)
        print("### GRADBOOST ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    elif classifier == "knn":
        clf, best_params = library.knn(X_val, y_val)
        clf.set_params(**best_params)
        print("### KNN ###")
        library.fit_report(clf, X_train, y_train, X_test, y_test)
    
    #elif classifier == "ensemble": 
        #ensemble
        #best_paramsSVM = gridSearchSVM(hyperOptTrainData, hyperOptLabel, cv)
        # best_paramsRF = gridSearchRF(hyperOptTrainData, hyperOptLabel, cv) 0.18-0.20
        #best_paramsGB = gridSearchGradientBoosting(hyperOptTrainData, hyperOptLabel, cv)
        #best_paramsAB = gridSearchAdaBoost(hyperOptTrainData, hyperOptLabel, cv)# 0.16-0.19 lr=0.1
        #best_paramsLR = gridSearchLogRegre(hyperOptTrainData, hyperOptLabel, cv)

        #pipeSVC = Pipeline([('scaler', StandardScaler()), ('svm', SVC(**best_paramsSVM, probability=True))])# false -> class
        # pipeRF  = Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier( **best_paramsRF,  n_jobs = -1, random_state=42))])
        #pipeAB  = Pipeline([('scaler', StandardScaler()), ('ab', AdaBoostClassifier( **best_paramsAB, random_state=42))]) 
        #pipeGB  = Pipeline([('scaler', StandardScaler()), ('gb', GradientBoostingClassifier( **best_paramsGB, random_state=42))])
        #pipeLR = Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(**best_paramsLR, random_state=42))]) 
        #pipeNB = Pipeline([('scaler', StandardScaler()), ('nb', GaussianNB())])

        #clf = VotingClassifier(estimators=[('svm', pipeSVC),
        #                                   ('lr', pipeLR),
        #                                   ('ab', pipeAB),
        #                                   ('gb', pipeGB),
        #                                   ('nb', pipeNB)],
        #                                   voting = 'soft')

        #fit_report(clf, X_train, y_train, X_test, y_test)

In [ ]:
!python --version